# Running on NERSC

Let's try to run `displayRank.jl` at NERSC. You should log into `cori.nersc.gov` now and we'll set things up.

## One time setup

Perform the following steps once to set things up

### Clone this git repository
In your NERSC home area, do 
```bash
git clone https://github.com/HighVelocityJuliaAnalysis/TryMPI.git
```

That will download this repository to your home area.

### Prepare Julia

We need to prepare Julia to install the packages we'll use. Do the following...

```bash
cd TryMPI/notebooks
julia --project
```

Once Julia starts, do the following in package mode (the `]` puts Julia in package mode).
```julia
]
instantiate
build --verbose
```

The `build --verbose` command will produce a bit of output. You should see a line with `Info: using systme MPI`. If you don't see that line, then something is wrong.

Exit Julia with Ctrl-d.

## Running interactively

`cd` back to `~/TryMpi/notebooks` if you aren't already there. 

On your laptop, we used `mpiexecjl` to run Julia scripts with MPI. On Cori, we instead use a command called `srun`. 

You are logged into a Cori *login* node. The login nodes are used to write code and prepare software. MPI programs will not run on the login node. You can see for yourself with,

```bash
srun -n 4 julia --project displayRank.jl
```

You'll likely get an error like,

```
srun: error: No architecture specified, cannot estimate job costs.
srun: error: Unable to allocate resources: Unspecified error
```

To run Julia with MPI interactively, we need to request an *interactive worker node*. The worker node is a Cori node used for running jobs and can run MPI. You can request a node with,

```bash
salloc --qos=interactive -C haswell --time=10 --nodes=1
```

You are requesting one *Haswell* node for 10 minutes of interactive use. Haswell is the type of machine you want. Cori has two choices, *Haswell* and *KNL*. The Haswell nodes are faster and have more memory, so we'll use that. 

It can take while for the `salloc` command to run. You should eventually someething like,

```
salloc: Pending job allocation 61073043
salloc: job 61073043 queued and waiting for resources
salloc: job 61073043 has been allocated resources
salloc: Granted job allocation 61073043
salloc: Waiting for resource configuration
salloc: Nodes nid00284 are ready for job
lyon@nid00284:~/TryMPI/notebooks> 
```

Note that your command prompt has changed. It doesn't say `cori` anymore. It will now say the name of the worker node (`nid00284` in this case). 

You may now try,

```bash
srun -n 4 julia --project displayRank.jl
```

You may request up to 64 ranks with the `-n` option. Try `calc_e_gather.jl` and `calc_e_reduce.jl` too. 

When you are done with the interactive worker node, you may type `exit` to go back to the Cori worker node. 

Or, when your 10 minutes are up, you will be logged out of the worker node automatially. You'll see a message like,

```
salloc: Job 61073043 has exceeded its time limit and its allocation has been revoked
```

If you want more time, issue the `salloc` command again. 





## Running batch jobs

For running big programs, we'll use batch jobs instead of interactive nodes. You will submit a *job* to a queue and the system will run your job when it can be scheduled. We will use the NERSC *debug* queue, which means we're limited in how many machinees we can ask for and how long we can run, but our jobs should start much more quickly (hopefully within a few minutes) after being submitted. We can get many more resources and run for longer from the regular queue, but it may take a long time for our jobs to start. 

NERSC, like most supercomputer centers, uses the *SLURM* system to manage jobs. SLURM has a series of commands to submit and check the status of jobs. See https://docs.nersc.gov/jobs/ for NERSC documentation. We'll do an example here.

The basic steps of submitting a job is,
1. Write a *job script* that is run on the worker nodes to execute your program. The job script is a shell script that uses special comments (e.g. a line with `#` in front) that are interpreted by SLURM to say how the job is to be run (putting this information in comments is a bit strange, but it's how SLURM works). 
2. Submit the script to SLURM with the `sbatch` command. 
3. Wait for the job to start running. 
4. When it starts, check the output file to look at what it's doing
5. When the job ends, check the output file again to look at what it did

We'll go through these steps.

### 1. Write the job script

Here is an example job script. You can copy and paste this into a file called `run_displayRank.sh`.

```bash
#!/bin/bash
#SBATCH --qos=debug
#SBATCH --time=5
#SBATCH --nodes=2
#SBATCH --tasks-per-node=10
#SBATCH --constraint=haswell
srun julia --project displayRank.jl
```

Note the `#SBATCH` comments. These will be read by the `sbatch` command to fill in options for SLUM to run your job. The parameters above mean,
- `--qos=debug` Run in the debug queue
- `--time=5` Run for no more than 5 minutes
- `--nodes=2` Run on two nodes
- `--tasks-per-node=10` Run on 10 ranks per node
- `--constraint=haswell` Use the Haswell machines

The `srun` command says to run `julia` with MPI. Note that we don't give the `-n` option to specify the number of ranks. We did that with `--nodes` and `--tasks-per-node` above in the `#SBATCH` options. In this case, we'll get 20 ranks (2 nodes with 10 ranks each). 

There are 64 cores (CPUs) on each node. So at most you can do `--tasks-per-node=64`. 

### 2. Run with `sbatch`

To submit the job to the batch system, do

```bash
sbatch run_displayRank.sh
```

Note that you can also *override* `#SBATCH` options in the job script by giving options to `sbatch`. For example,

```bash
# Override example
sbatch --nodes=3 run_displayRank.sh
```

will override the `#SBATCH --nodes=2` in the job script and will instead run on 3 nodes (so you'll have 30 ranks insteaed of 20). 

### 3. Wait for the job to start

Do,

```bash
sqs
```

To see the status of your job. Mine started running right away, so I saw,

```
JOBID            ST USER      NAME          NODES TIME_LIMIT       TIME  SUBMIT_TIME          QOS             START_TIME           FEATURES       NODELIST(REASON
61090270         R  lyon      run_displayR  2           5:00       0:03  2022-07-14T14:53:19  debug_hsw       2022-07-14T14:53:20  haswell        nid00[754-755] 
```

The `ST` column is the status. If it says `R` that means your job is running. If it says `PD`, that means your job is pending (not running yet), probably because you are waiting for the number of nodes you want to be freed up. If you only see the header and no data, that means your job is probably already done! 

If for some reason you want to cancel your job, whether it is running or pending, you can do

```bash
scancel JOBID  # Where JOBID is the numerical job id given by sqs
```

Your job will stop and will be removed from the queue eventually (if you do `sqs` after canceling a job, you may see your job with status `CA`, which means it is being canceled).

See this [list of job status codes](https://slurm.schedmd.com/sacct.html#SECTION_JOB-STATE-CODES).

### 4. Monitor job output

When your job is running, a file will be created in the directory from where you ran `sbatch` called `slurm-XX.out` where `XX` is the JOBID. So in my case, I got `slurm-61090270.out`. You can look at this file to see output from the job in real time. If you want to keep watching the file, do

```bash
tail -F slurm-XX.out   # Replace XX with the JOBID
```

and you will watch the file as it grows. Press `Ctrl-c` to exit watching the file and go back to the prompt.

### 5. Look at the job output when it ends

When `sqs` no longer shows your job, that means it is done. Look at the output file (mentioned above) to see what it did. You can do 

```bash
less slurm-XX.out  # Replace XX with the JOBID
```

If you forget the Job ID, just do an `ls` of the directory. The `slurm-XX.out` file with the largest Job ID is the last one that ran (or is running). 

You can also ask SLURM about your job history. You can do,

```bash
sacct
```

For a job that I ran, I got the following,

```
JobID           JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
61090270     run_displ+  debug_hsw      m3249        128  COMPLETED      0:0 
61090270.ba+      batch                 m3249         64  COMPLETED      0:0 
61090270.ex+     extern                 m3249        128  COMPLETED      0:0 
61090270.0        julia                 m3249        128  COMPLETED      0:0 
```

Although I ran one job, there are four entires. Note that the JobID number is the same for each four. SLURM keeps track of each *job step*. To execute `run_displayRank.sh`, SLURM did (each step below is one entry above)...

- Set up the job in the debug queue on 2 nodes (what I asked for). There are 64 CPUs per node, so 2 nodes is 128 CPUs. 
- Ran my job script (batch) on one node (64 CPUs)
- The `srun` command in the job script says to run an external program (extern). SLURM set that up on 2 nodes.
- Finally, the `julia` command (what `srun` ran) executed on those 2 nodes. 

The command by default doesn't say how long the job ran for. You can add this by doing the following (this makes for more sensical output)...

```bash
sacct --format=JobID,JobName,NNodes,NTasks,CPUTime,Elapsed
```

For my job, I got this output,

```
JobID           JobName   NNodes   NTasks    CPUTime    Elapsed 
------------ ---------- -------- -------- ---------- ---------- 
61090270     run_displ+        2            00:08:32   00:00:04 
61090270.ba+      batch        1        1   00:04:16   00:00:04 
61090270.ex+     extern        2        2   00:08:32   00:00:04 
61090270.0        julia        2       20   00:06:24   00:00:03 
```

Here, `NTasks` is the total number of MPI Ranks that were running. Again you see the four entries per job. The elapsed time for the whole job is given by the first entry - 4 seconds. That's "Wall clock time" or the actual time it took. The `CPUTime` is the total time for all CPUs on the job. 

You see that the total CPU time is 8 minutes 32 seconds for a job that lasted all of 4 seconds. Why is the CPU time so big? 8m32s is 512 seconds. When you run on a Cori node, you are allocated ALL of the CPUs on that node, whether you use them or not. Since we had 128 CPUs total (2 nodes $\times$ 64 processors each), $512/128 = 4$ s. 






